In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools 
import multiprocessing as mp
import numpy as np
import sys

sys.path.append('../')
from baseModels.SVM.model import SVM

In [2]:
feature_dir = '../featEngg/ifeatMethods/data/featvec/trainfiles/'
feature_files = [f.name for f in os.scandir(feature_dir) if f.name.endswith('.csv.gz')]

In [3]:
label_file = '../data/LabelFiles/EnzymeLabelsMultiClass.csv'

In [4]:
def model_evaluate(filename,random_seed):
    df1 = pd.read_csv(feature_dir+filename,header=None)
    df2 = pd.read_csv(label_file,header=None)
    df = df1.merge(df2,on=0)
    enz_names = df[0].values
    X = df.iloc[:,1:-1].values
    y = df.iloc[:,-1].values
    if X.shape[1]<55:
        n_comp = int(0.75*X.shape[1])
    else:
        n_comp=55
    X_train, X_valid, y_train, y_valid,enz_train,enz_valid = train_test_split(X, y,enz_names, test_size=0.25, random_state=random_seed)
    svm = SVM(X_train,X_valid,y_train,y_valid,verbose=False,optimize=False, pca_comp=n_comp,kern='rbf',regC=30)
    return svm.acc_train,svm.acc_valid
 

In [5]:
mp.cpu_count()

24

In [6]:
pool = mp.Pool(mp.cpu_count())

In [7]:
iterable = list(itertools.product(feature_files,range(10000)))

In [8]:
%%time
model_accs = list(pool.starmap(model_evaluate,iterable))

CPU times: user 437 ms, sys: 77.7 ms, total: 515 ms
Wall time: 20min 47s


In [9]:
mydict = dict()

for m,acc in zip(iterable,model_accs):
    model_name=m[0].replace('.csv.gz','')
    seed = m[1]
    if model_name not in mydict:
        mydict[model_name] = {seed:acc}
    else:
        mydict[model_name][seed] = acc

In [10]:
def get_model_stats(mname):
    train_accs = [v[0] for k,v in mydict[mname].items()]
    test_accs = [v[1] for k,v in mydict[mname].items()]
    return {'train_min':min(train_accs),'train_mean':np.mean(train_accs),
            'train_max':max(train_accs),'train_std':np.std(train_accs),
            'test_min':min(test_accs),'test_mean':np.mean(test_accs),
            'test_max':max(test_accs),'test_std':np.std(test_accs)}

In [11]:
get_model_stats('Geary')

{'train_min': 0.9069767441860465,
 'train_mean': 0.9498616279069769,
 'train_max': 1.0,
 'train_std': 0.015307460040210446,
 'test_min': 0.4827586206896552,
 'test_mean': 0.7840068965517242,
 'test_max': 1.0,
 'test_std': 0.07165953290638638}

In [12]:
results_dict= {mname:get_model_stats(mname) for mname in mydict.keys()}
    

In [13]:
results_df = pd.DataFrame(results_dict).T

In [14]:
results_df.to_csv('../data/SimResults/iFeatResults.csv',index=True)

In [15]:
results_df.sort_values('train_mean')

,train_min,train_mean,train_max,train_std,test_min,test_mean,test_max,test_std
GAAC,0.697674,0.811591,0.930233,0.029733,0.275862,0.585907,0.896552,0.082592
SOCNumber,0.860465,0.930264,0.988372,0.017132,0.413793,0.753686,1.000000,0.077702
CTDC,0.872093,0.936021,0.988372,0.014832,0.413793,0.740000,1.000000,0.079586
PAAC,0.883721,0.938833,1.000000,0.016400,0.379310,0.711921,0.965517,0.079960
AAC,0.883721,0.940809,1.000000,0.016079,0.310345,0.701638,0.965517,0.082040
CTriad,0.906977,0.941178,1.000000,0.013413,0.448276,0.789079,1.000000,0.072140
KSCTriad,0.906977,0.941178,1.000000,0.013413,0.448276,0.789079,1.000000,0.072140
APAAC,0.895349,0.943726,1.000000,0.015135,0.379310,0.707476,0.965517,0.081319
CTDT,0.895349,0.946341,1.000000,0.015470,0.379310,0.708338,0.931034,0.079850
QSOrder,0.906977,0.946362,1.000000,0.014989,0.379310,0.732659,1.000000,0.079713
